In [1]:
from queryAnalyzer import getPlayersInQuery, getLeaguesInQuery, getTeamsInQuery, getSeasonsInQuery, getStagesInQuery
import sqlAnswers

In [2]:
import sqlite3
sqlite_file = 'database.sqlite'

query1 = "Thomas Mueller ist der beste Spieler der Welt!"
query1 = "Müller Thomas ist der beste Spieler der Welt, nicht Aaron Doran!"
query2 = "Bundesliga und Jupiler League sind supi!"
query2 = "Die deutsche Liga ist supi!"


In [3]:
#make query insensitive for umlaute
def preProcessQuery(query):
    umlaut_a = 'ä'.encode()
    umlaut_A = 'Ä'.encode()
    umlaut_o = 'ö'.encode()
    umlaut_O = 'Ö'.encode()
    umlaut_u = 'ü'.encode()
    umlaut_U = 'Ü'.encode()
    
    query = query.encode()
    
    query = query.replace(umlaut_a, b'ae')
    query = query.replace(umlaut_A, b'Ae')
    query = query.replace(umlaut_o, b'oe')
    query = query.replace(umlaut_O, b'Oe')
    query = query.replace(umlaut_u, b'ue')
    query = query.replace(umlaut_U, b'Ue')
    
    query = query.decode('utf-8')
    
    if(query[-1:] == '?' or query[-1:] == '!' or query[-1:] == '.' ):
        return query[:-1]
    return query

Query wird verarbeitet:
    
- Liefert eine playerID eines Spielers zurück
- Keyword "wiegt" (oder auch "schwer") wird gefunden

-> dieses keyword hat nen pointer auf eine SQL Query mit Platzhaltern, in welches die ID des Spielers von dem gesetzt wird


'SELECT weight FROM player WHERE id = playerID'


flair tool 
was passiert wenn der Input da rein kommt

Queries mit "Tags"

Spieler
Ligen
Verein (evtl abkürzung)
Saison / Jahr





In [4]:
from flair.embeddings import WordEmbeddings, FlairEmbeddings, StackedEmbeddings

# create a StackedEmbedding object that combines crawl and forward/backward flair embeddings 
#(which take the words next to it into account before giving a vector)
german_embedding = WordEmbeddings('de-crawl')
#german_embedding = StackedEmbeddings([
 #                                       WordEmbeddings('de-crawl'),
  #                                      FlairEmbeddings('de-forward'),
   #                                     FlairEmbeddings('de-backward'),
    #                                   ])
#needed to check for unknown (e.g. non german) or wrong spelled words
crawl_embedding = WordEmbeddings('de-crawl')
from flair.data import Sentence
 
import numpy
import json

In [5]:
#returns the nearest vector from the query to the individual training sentences of a specific index
def getVectorOfAnswer(index, queryV):
    print(index)
    #with open('trainData.json', 'r') as myfile:
    #    data=myfile.read()
    with open('trainData.json', encoding='utf-8') as myfile:
        obj = json.load(myfile)
    
    
    #obj = json.loads(data)
    bestEmbedding = 0
    inIteration1 = True
    
    
    trainDataIndex = str(index)
    
    print("folgende antworten werden angezapft")
    
    for trainingSentence in obj[trainDataIndex]:
        
        #trainingSentence =  replaceKeywords(preProcessQuery(trainingSentence))[1]
        
        #print(trainingSentence)
        trainingSentence =  preProcessQuery(trainingSentence)
        #print(trainingSentence)
        #trainingSentence = replaceKeywords(trainingSentence)[1]
        #print(trainingSentence)
        
        embed = getVectorOfQuery(trainingSentence)
        
        
        if(inIteration1 == True):
            bestEmbedding = embed
        elif(angle_between(queryV, bestEmbedding) > angle_between(queryV, embed)): #if a better trining sentence was found
            bestEmbedding = embed
        inIteration1 = False;
            
    return bestEmbedding
#print(getVectorOfAnswer(0))

In [6]:
import numpy as np
def replaceKeywords(query):
    pQ = getPlayersInQuery(query)
    lQ = getLeaguesInQuery(pQ[1])
    tQ = getTeamsInQuery(lQ[1])
    sQ = getSeasonsInQuery(tQ[1])
    dQ = getStagesInQuery(sQ[1])
    return dQ


def getVectorOfQuery(query):
    sentence = Sentence(query)
    german_embedding.embed(sentence)
    
    embed = 0
    for token in sentence:
        #print(str(token) + ": ")
        #print(np.linalg.norm(token.embedding.numpy()))
        embed += token.embedding.cpu().numpy()
    return embed


def unit_vector(vector):
    return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
    v1_u = unit_vector(v1)
    v2_u = unit_vector(v2)
    return np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0))



In [7]:
def containsWrongWords(query):
    sentence = Sentence(query)
    crawl_embedding.embed(sentence)
    
    for token in sentence:
        print(token)
        #print(np.linalg.norm(token.embedding.numpy()))
        if(np.linalg.norm(token.embedding.cpu().numpy()) == 0.0):
            return True
    return False
    

In [8]:
def getAnswer(question):
    query = preProcessQuery(question)
    
    pQ = getPlayersInQuery(query)
    lQ = getLeaguesInQuery(pQ[1])
    tQ = getTeamsInQuery(lQ[1])
    sQ = getSeasonsInQuery(tQ[1])
    dQ = getStagesInQuery(sQ[1])
    #print("Query, aus dem der Vektor gemacht wird:")
    print(dQ[1]) # die letzt query die dann auch dem vector gegeben wird
    print("")
    
    #if(containsWrongWords(yQ[1])):
    #    return "Deine Antwort enthält ein oder mehrer unbekannte Wörter!"
    
    queryV = getVectorOfQuery(dQ[1]) # erwarter preprocessed Query, TODO 
    
    queryRefined = [pQ[0],lQ[0],tQ[0],sQ[0],dQ[0]]
    #queryV = getVectorOfQuery(pQ[1]) # erwarter preprocessed Query, TODO 
    #print(getPlayersInQuery(query)[1])

    #queryRefined = [pQ[0], getLeaguesInQuery(query),getTeamsInQuery(query),getSeasonsInQuery(query),getStagesInQuery(query)]
    print("gefundene Dinge: ")
    print(queryRefined)
    answers = sqlAnswers.getAnswers()
    #answers = getAnswers()


    queryToCompare = [len(queryRefined[0]),len(queryRefined[1]),len(queryRefined[2]),len(queryRefined[3]),len(queryRefined[4])]
    possibleAnswers = [] 
    #contains index with possible ansers which could match the query depending
    #on number of player occurances etc

    for x in range(0,len(answers)): #check for every "answer", if [1,0,0,0] matches [1,0,0,0] etc
        sqlMatches = True;
        for i in range(0, len(queryToCompare)): #interate through the whole [0,1,2,3] array and check 
            if(queryToCompare[i] != answers[x][i]):  # if something doesnt match, throw it out
                sqlMatches = False
        if(sqlMatches):
            possibleAnswers.append(x)
        

    print("In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: ")       
    print(possibleAnswers)
    if(len(possibleAnswers) == 1):
        return answers[possibleAnswers[0]][5](queryRefined)
    if(len(possibleAnswers) == 0):
        return "Es konnte leider keine passende Antwort gefunden werden!"
    angles = []
    for i in possibleAnswers:
        angles.append(angle_between(getVectorOfAnswer(answers[i][6], queryV),queryV))#answers[answerIndex][6] equals the index of the train data json

    print(angles)
    print("Beste gefundene Antwort anhand der Cosinus Ähnlichkeit: " + str(possibleAnswers[angles.index(min(angles))]))

    indexOfNeededAnswer = possibleAnswers[angles.index(min(angles))];
    print(indexOfNeededAnswer)
    
    return answers[indexOfNeededAnswer][5](queryRefined)

In [9]:
#Stellen Sie Ihre Frage:
#gebursdatum gewicht fuß Verein VereinInSaision
# Wie war die Aufstellung des FC Bayern Munich am 15.Spieltag der Saison 2012/2013? 
# liefert 10 11 12 14 als antworten vom Index her -> Stimmt
# bei 10 Muss es zB auf Training Data 8 Zeigen - Händisch EIntragen TODO
eingabe = input("")
print(getAnswer(eingabe))

Was ist der bevorzugte Fuß von Manuel Neuer?
Was ist der bevorzugte Fuß von SPIELER

gefundene Dinge: 
[[6556], [], [], [], []]
In Frage kommende Anworten von der Anzahl der gefundenen Namen etc her: 
[0, 1, 2, 3, 4]
0
folgende antworten werden angezapft
Wann wurde SPIELER geboren
Wann hat SPIELER Geburtstag
Wie alt ist SPIELER
Was ist das Geburtsdatum von SPIELER
Wann ist SPIELER geboren
Wann ist SPIELER auf die Welt gekommen
Wann ist SPIELER geboren
Geburtsdatum SPIELER
SPIELER Geburtstag
Bitte sag mir, wann SPIELER geboren wurde
Wie lautet das Geburtsdatum von SPIELER
Wann ist SPIELERs Geburtstag
1
folgende antworten werden angezapft
Wie schwer ist SPIELER
Wie viel wiegt SPIELER
Wie viel bringt SPIELER auf die Waage
Welches Gewicht besitzt SPIELER
Gewicht SPIELER
Wie viel kg wiegt SPIELER
Was ist das Gewicht von SPIELER
SPIELER Gewicht
Wie viel wiegt SPIELER (in Kilogramm)
was wiegt thomas mueller
Was ist SPIELER's Gewicht
Wie ist das Gewicht von SPIELER
6
folgende antworten werden 

Manuel Neuer und Thomas Müller 2009 Bundesliga

    Funktioniert:
    
Wo spielte Thomas Müller Heimverein?
Was ist der Heimverein von Manuel Neuer?
Was ist der Heimverein von Toni Kroos?

Was ist das Geburtsdatum von Jerome Boateng?

was ist der bevorzugte Fuß von Toni Kroos?

Wo hat Manuel Neuer 2008/2009 gespielt?
Wo hat Manuel Neuer 2014/2015 gespielt?
    
In welcher Saison spielte Mario Götze beim FC Bayern Munich?

Wie hat der FC Bayern Munich am 15.Spieltag 2012/2013 gespielt?
Wie hat der FC Bayern Munich am 15.Spieltag gespielt?

Gegen wen hat der FC Bayern Munich am 15.Spieltag 2012/2013 gespielt?
Gegen wen hat der FC Bayern Munich am 15.Spieltag gespielt?


    SQL sollte funktionieren, aber problematisch:

Wie groß ist Toni Kroos? -> nimmt random andere Sachen, die nur 1 Spieler fordern

Wie war die Aufstellung des FC Bayern Munich am 15.Spieltag der Saison 2012/2013? -> nimmt getWeight
Wie war die Aufstellung des FC Bayern Munich am 15.Spieltag? -> nimmt birthday

Wie viele Siege/Niederlagen hatte der FC Bayern Munich (gegen Dortmund) (in der Saison 2012/13)?

War der 15.Spieltag der Saison 2012/2013 ein Heimspiel für FC Bayern Munich?

In [10]:
query = preProcessQuery("In welchem Verein spielt SPIELER SAISON?")
    
pQ = getPlayersInQuery(query)
lQ = getLeaguesInQuery(pQ[1])
tQ = getTeamsInQuery(lQ[1])
sQ = getSeasonsInQuery(tQ[1])
dQ = getStagesInQuery(sQ[1])
queryV = getVectorOfQuery(dQ[1])


print(getVectorOfAnswer(13, queryV))


13
folgende antworten werden angezapft
An welchem Spieltag hat TEAM in Saison SEASON gegen TEAM gespielt
wann hat der TEAM in der saison SEASON gegen TEAM gespielt
Wann hat der TEAM in der Saison SEASON gegen den TEAM gespielt
[-4.33200002e-01 -1.94299996e-01  1.19299993e-01  1.85499996e-01
  1.95999980e-01  1.14980006e+00  2.19300002e-01  9.87000018e-02
 -4.31600004e-01 -2.08000168e-02 -3.18199992e-01 -1.28300011e-01
 -2.90300012e-01 -3.52699995e-01  3.28599989e-01 -1.06599994e-01
  4.39300001e-01 -2.81700015e-01 -7.68000036e-02 -3.65699977e-01
 -1.66599989e-01  3.32300007e-01  1.29899994e-01  3.10799986e-01
 -5.80600023e-01 -9.91999805e-02 -4.36000079e-02  1.15399994e-01
  2.01099992e-01 -2.07300037e-01 -5.87400019e-01 -2.73300022e-01
 -6.00899994e-01 -2.10499987e-01 -3.65000069e-02 -3.41000035e-02
  8.57999995e-02  2.04000138e-02 -2.10200012e-01 -5.12000062e-02
 -6.48000166e-02 -5.50599933e-01 -2.68600017e-01 -2.45300025e-01
 -3.71900022e-01  4.40700024e-01  1.56899989e-01  2.047000